In [1]:
# import dependencies
import requests
# Data Cleaning
import pandas as pd 
import openpyxl
# SQL Datbase Adapter and ORM
#%pip install psycopg2
import psycopg2
from sqlalchemy import create_engine, text
# Visualizations
import matplotlib.pyplot as plt
import numpy as np
from numpy import polyfit
from scipy import stats

In [2]:
#format list for the dataframe
key = '6924a24c5e8fd68ab4c75b8911dde245'
state_ids = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL',
             'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
             'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE',
             'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
             'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
             'VA', 'WA', 'WI', 'WV', 'WY']

In [3]:
#create dataframe and read in the two json files holding personal expenditures and personal income per capita
for x in state_ids:
  RINCPC = f"https://api.stlouisfed.org/fred/series/observations?series_id={x}RPIPC&api_key={key}&file_type=json"
  response = requests.get(RINCPC)
  series = pd.DataFrame(response.json()["observations"])
  series["value"] = series.rename(columns={'value': x}, inplace=True)
  series = series[['date', x]]
  if x == state_ids[0]:
    df = series
  else:
    df = pd.merge(df, series, on='date', how='inner')
df = df.set_index('date').T
df = df.reset_index()
df['region/state of residence'] = df['index']
df = df.drop(columns='index')
states = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
          'FL': 'Florida','GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'kentucky',
          'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 
          'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska','NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 
          'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania',
          'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
          'VA': 'Virginia', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming'}
df = df.rename(columns= {'2008-01-01': '2008', '2009-01-01': '2009',  '2010-01-01': '2010', '2011-01-01': '2011', '2012-01-01': '2012', '2013-01-01': '2013',
         '2014-01-01': '2014', '2015-01-01': '2015',  '2016-01-01': '2016', '2017-01-01': '2017', '2018-01-01': '2018', '2019-01-01': '2019',
         '2020-01-01': '2020', '2021-01-01': '2021', '2022-01-01': '2022'})         
df = df.replace(states)


In [ ]:
#print dataframe to csv - real income - has income per capita income per state
df.to_csv('Data/realincome.csv', encoding='utf-8', index=False)

In [ ]:
income_csv = pd.read_csv('Data/realincome.csv')
income_csv.head()

In [ ]:
cms_wb = openpyxl.load_workbook('Data/Residence_all_tables.xlsx')
Ws = cms_wb.sheetnames

In [ ]:
print(Ws)

In [ ]:

df2 = pd.read_excel('Data/Residence_all_tables.xlsx', 'Table 11 Personal Health Care')
df3 = pd.read_excel('Data/Residence_all_tables.xlsx', 'Table 12 Hospital')
df4 = pd.read_excel('Data/Residence_all_tables.xlsx', 'Table 13 Physician and Clinics')
df5 = pd.read_excel('Data/Residence_all_tables.xlsx', 'Table 16 Home Health')
df6 = pd.read_excel('Data/Residence_all_tables.xlsx', 'Table 17 Nursing')
df7 = pd.read_excel('Data/Residence_all_tables.xlsx', 'Table 23 Medicare Per Enrollee')
df8 = pd.read_excel('Data/Residence_all_tables.xlsx', 'Table 26 Medicaid Per Enrollee')
df9 = pd.read_excel('Data/Residence_all_tables.xlsx', 'Table 29 Private Per Enrollee')

name2 = 'Personal Health Care'
name3 = 'Hospital'
name4 = 'Physician and Clinics'
name5 = 'Home Health'
name6 = 'Nursing'
name7 = 'Medicare Per Enrollee'
name8 = 'Medicaid Per Enrollee'
name9 = 'Private Per Enrollee'

def clean(data):
    dat = data
    dat.columns = (dat.iloc[0])
    dat = dat.drop(62)
    dat = dat.drop(61)
    dat = dat.drop(0)
    dat = dat.rename(columns= {2008.0: 2008, 2009.0: 2009, 2010.0: 2010, 2011.0: 2011, 2012.0: 2012, 2013.0: 2013, 2014.0: 2014,
                           2015.0: 2015, 2016.0: 2016, 2017.0: 2017, 2018.0: 2018, 2019.0: 2019, 2020.0: 2020})
    dat = dat[['Region/state of residence', 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]]
    dat = dat.astype({2008: int, 2009: int, 2010: int, 2011: int, 2012: int, 2013: int, 2014: int,
                           2015: int, 2016: int, 2017: int, 2018: int, 2019: int, 2020: int})
    dat = dat.rename(columns={'Region/state of residence': 'region/state of residence'})
    if data.equals(df2):
        dat.to_csv(f'Data/{name2}.csv', encoding='utf-8', index=False)
    elif data.equals(df3):
        dat.to_csv(f'Data/{name3}.csv', encoding='utf-8', index=False)
    elif data.equals(df4):
        dat.to_csv(f'Data/{name4}.csv', encoding='utf-8', index=False)
    elif data.equals(df5):
        dat.to_csv(f'Data/{name5}.csv', encoding='utf-8', index=False)
    elif data.equals(df6):
        dat.to_csv(f'Data/{name6}.csv', encoding='utf-8', index=False)
    elif data.equals(df7):
        dat.to_csv(f'Data/{name7}.csv', encoding='utf-8', index=False)
    elif data.equals(df8):
        dat.to_csv(f'Data/{name8}.csv', encoding='utf-8', index=False)
    elif data.equals(df9):
        dat.to_csv(f'Data/{name9}.csv', encoding='utf-8', index=False)
    return dat

clean(df2), clean(df3), clean(df4), clean(df5), clean(df6), clean(df7), clean(df8), clean(df9)

In [ ]:
user = 'postgres' # Your username (try entering pgadmin to find)
password = '1234' # Your Password
host = 'localhost'
port = '5432'
database_name = 'Healthcare_DB'

connection = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database_name}'
engine = create_engine(connection)
with engine.connect() as connection:
    query = text("SELECT * FROM realincome;")
    query2 = text("SELECT * FROM medicaid;")
    print(df2)
    df2 = pd.read_sql(query2, connection)
    df = pd.read_sql(query, connection)

connection.close()

df2_mean = df2.drop(columns={'region/state_of_residence'})
df2['medicaid'] = df2_mean.mean(axis=1)
df2.index = df2['region/state_of_residence']
df2 = df2.drop(index={'New England', 'District of Columbia', 'Great Lakes', 'Mideast', 'United States', 'Southeast',
                     'Southwest', 'Rocky Mountains', 'Far West'})
df2 = df2.sort_index()
df2 = df2['medicaid']

df_mean = df.drop(columns={'region/state_of_residence'})
df['realincome'] = df_mean.mean(axis=1)
df.index = df['region/state_of_residence']
df = df[['realincome', '2020']]
df = df.merge(df2, on='region/state_of_residence')
df = df.reset_index('region/state_of_residence')
print(df.head())

opt_out = ['Wyoming', 'Texas', 'South Dakota', 'Wisconsin', 'Mississippi', 'Tennessee', 'Alabama', 'Georgia', 'North Carolina',
        'South Carolina', 'Kansas', 'Florida']
upon_adoption = ['Arizona',	'Arkansas',	'California', 'Colorado', 'Connecticut', 'Delaware', 'Hawaii', 'Illinois', 'Iowa',
                 'Kentucky', 'Maryland', 'Massachusetts', 'Minnesota', 'Nevada', 'New Jersey', 'New York', 'North Dakota',
                 'Ohio', 'Rhode Island', 'Vermont', 'Washington', 'West Virginia']

colors_dict = {'Yes': 'red', 'No': 'blue', 'Adopters': 'green'}

plt.figure(figsize=(15, 15))
colors = [None] * len(df)
for i, row in df.iterrows():
    state = row['region/state_of_residence']
    if state in opt_out:
        colors[i] = 'red'
        label = 'Opt Out'
    elif state in upon_adoption:
        colors[i] = 'blue'
        label2 = 'Original Adopter'
    else:
        colors[i] = 'green'
        label3 = 'Adopted Later'

plt.scatter(df['realincome'], df['medicaid'], s=50, c=colors, alpha=1.0, label=label)
plt.scatter(df['realincome'], df['medicaid'], s=50, c='blue', alpha=0.2, label=label2)
plt.scatter(df['realincome'], df['medicaid'], s=50, c='green', alpha=0.2, label=label3)
m, b = polyfit(df['realincome'], df['medicaid'], 1)
r, p = stats.pearsonr(df['realincome'], df['medicaid'])
r_squared = r**2
x_trend = np.linspace(min(df['realincome']), max(df['realincome']), 100)
y_trend = m * x_trend + b
plt.plot(x_trend, y_trend, color='red', label='Trendline')
plt.annotate(f"R² = {r_squared:.2f}", xy=(0.8, 0.8), xycoords='figure fraction')


for i, (inc, home_health, region) in enumerate(zip(df['realincome'], df['medicaid'], df['region/state_of_residence'])):
    plt.text(inc, home_health + 0.1, region, ha='center', va='bottom', fontsize=10)

plt.xlabel('Personal Income Expenditure per Capita ($)')
plt.ylabel('Medicaid Expenditure per Capita ($)')
plt.title('Medicaid Expenditure by State (2008-2020)')
plt.legend(title='Medicaid Expansion Status')
plt.grid(True)
plt.show()

In [ ]:
user = 'postgres'
password = '1234' 
host = 'localhost'
port = '5432'
database_name = 'Healthcare_DB'

connection = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database_name}'
engine = create_engine(connection)
with engine.connect() as connection:
    query = text("SELECT * FROM personal_health_care_regions;")
    query2 = text("SELECT * FROM regional_averages_income;")
    df_average_income = pd.read_sql(query2, connection)
    df_healthcare_expen = pd.read_sql(query, connection)

with engine.connect() as connection:
    df_average_income = pd.read_sql(query2, connection)
    df_healthcare_expen = pd.read_sql(query, connection)
    
connection.close()

df_healthcare_expen.rename(columns={'region/state_of_residence': 'Region'}, inplace=True)
df_average_income.rename(columns={'region': 'Region'}, inplace=True)

df_healthcare_expen

In [ ]:
income_cost_percentage = pd.DataFrame()

for year in range(2008, 2021):
    income_cost_percentage[f'{year}'] = (df_healthcare_expen[f'{year}'] / df_average_income[f'{year}']) * 100

income_cost_percentage.insert(0, 'Region', df_healthcare_expen['Region'])

income_cost_percentage.sort_values(by='Region', inplace=True)

income_cost_percentage

In [ ]:
for region in df_healthcare_expen['Region']:
    plt.plot(df_healthcare_expen.columns[1:], df_healthcare_expen[df_healthcare_expen['Region'] == region].iloc[:, 1:].values.flatten(), label=region)

plt.title('Annual Expenditure on Personal Healthcare by Region (2008-2020)')
plt.xlabel('Year')
plt.ylabel('Annual Expenditure per Capita')
plt.xticks(df_healthcare_expen.columns[1:], rotation=45)
plt.legend(title='Region', bbox_to_anchor=(1, 1), loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
for region in df_average_income['Region']:
    plt.plot(df_average_income.columns[1:], df_average_income[df_average_income['Region'] == region].iloc[:, 1:].values.flatten(), label=region)

plt.title('Average Annual Income by Region (2008-2020)')
plt.xlabel('Year')
plt.ylabel('Average Annual Income per Capita')
plt.xticks(df_average_income.columns[1:], rotation=45)
plt.legend(title='Region', bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In [ ]:
for region in income_cost_percentage.iterrows():
    plt.plot(income_cost_percentage.columns[1:], region[1][1:], label=region[1][0])

plt.title('% of Income Spent on Personal Healthcare (2008-2020)')
plt.xlabel('Year')
plt.ylabel('% of Income Spent on Personal Healthcare per Capita')
plt.xticks(rotation=45)
plt.legend(title='Region', bbox_to_anchor=(1, 1), loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
#establish a connection
user = 'postgres'
password = '1234' 
host = 'localhost'
port = '5432'
database_name = 'Healthcare_DB'

connection = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database_name}'
engine = create_engine(connection)
with engine.connect() as connection:
    comparisons_df = pd.read_sql_table(
    "Comparisons",
    con=engine,
    columns=['region/state_of_residence', 'Medicare_2010', 'PrivateIns_2010', 'Medicaid_2010', 
             'Medicare_2015', 'PrivateIns_2015', 'Medicaid_2015', 'Medicare_2020', 'PrivateIns_2020', 'Medicaid_2020'],)
connection.close()

In [ ]:
#print the postgresql table loaded as pandas dataframe
sorted_comparisons=comparisons_df.sort_values('region/state_of_residence')
sorted_comparisons.head(60)

In [ ]:
print(sorted_comparisons.columns)

In [ ]:
sorted_comparisons.set_index(sorted_comparisons.columns[0])

In [ ]:
sorted_comparisons.plot(x='region/state_of_residence', y=["Medicare_2010", "PrivateIns_2010", "Medicaid_2010"],
                xlabel='State',
                ylabel="Dollars per Enrollee", 
                title="Insurance Costs Per Enrollee in 2010",
                figsize=(20,5), colormap="viridis")
                  
sorted_comparisons.plot(x="region/state_of_residence", y=["Medicare_2015", "PrivateIns_2015", "Medicaid_2015"],
                xlabel="State",
                ylabel="Dollars per Enrollee", 
                title="Insurance Costs Per Enrollee in 2015",
                figsize=(20,5), colormap="viridis")

sorted_comparisons.plot(x="region/state_of_residence", y=["Medicare_2020", "PrivateIns_2020", "Medicaid_2020"],
                xlabel="State",
                ylabel="Dollars per Enrollee", 
                title="Insurance Costs Per Enrollee in 2020",
                figsize=(20,5), colormap="viridis")

plt.show()

In [ ]:
sorted_comparisons.plot(x='region/state_of_residence', y=["Medicare_2010", "PrivateIns_2010", "Medicaid_2010"],
                xlabel='State',
                ylabel="Dollars per Enrollee", 
                title="Insurance Costs Per Enrollee in 2010",
                figsize=(20,5), colormap="Paired")

sorted_comparisons.plot(x="region/state_of_residence", y=["Medicare_2010", "PrivateIns_2010", "Medicaid_2010"],
                xlabel="State",
                ylabel="Dollars Per Enrollee", kind="bar", stacked=True, 
                title="Insurance Costs Per Enrollee in 2010",
                width = .7, figsize=(20,5),colormap='Paired')

plt.show

In [ ]:
sorted_comparisons.plot(x="region/state_of_residence", y=["Medicare_2015", "PrivateIns_2015", "Medicaid_2015"],
                xlabel="State",
                ylabel="Dollars per Enrollee", 
                title="Insurance Costs Per Enrollee in 2015",
                figsize=(20,5), colormap="cool")

sorted_comparisons.plot(x="region/state_of_residence", y=["Medicare_2015", "PrivateIns_2015", "Medicaid_2015"],
                xlabel="State",
                ylabel="Dollars Per Enrollee", kind="bar", stacked=True, 
                title="Insurance Costs Per Enrollee in 2015",
                width = .7, figsize=(20,5),colormap='cool')

plt.show

In [ ]:
sorted_comparisons.plot(x="region/state_of_residence", y=["Medicare_2020", "PrivateIns_2020", "Medicaid_2020"],
                xlabel="State",
                ylabel="Dollars per Enrollee", 
                title="Insurance Costs Per Enrollee in 2020",
                figsize=(20,5), colormap="winter")

sorted_comparisons.plot(x="region/state_of_residence", y=["Medicare_2020", "PrivateIns_2020", "Medicaid_2020"],
                xlabel="State",
                ylabel="Dollars Per Enrollee", kind="bar", stacked=True, 
                title="Insurance Costs Per Enrollee in 2020",
                width = .7, figsize=(20,5),colormap='winter')

plt.show

In [ ]:
# Database connection parameters
host = "localhost"
port = "5432"
dbname = "Healthcare_DB"
user = "postgres"
password = "1234"

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)

# Query to fetch data from the table
query = "SELECT * FROM Combined_2020"

# Load data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df.head())

df.to_csv('Data/Combined_2020.csv', encoding='utf-8', index=False)

In [ ]:
from math import pi
import seaborn as sns

# Load the CSV file into a DataFrame
file_path = 'Data/Combined_2020.csv'
data = pd.read_csv(file_path)

# Prepare data for radar chart (example for 2020)
categories = data['region/state of residence']
n_categories = len(categories)

home_health_2020 = data['HomeHealth_2020'].values
nursing_2020 = data['Nursing_2020'].values

# Add the first value to the end to close the circle
home_health_2020 = np.append(home_health_2020, home_health_2020[0])
nursing_2020 = np.append(nursing_2020, nursing_2020[0])

# Compute angle for each category
angles = [n / float(n_categories) * 2 * pi for n in range(n_categories)]
angles += angles[:1]

# Initialize the radar chart
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

# Draw one axe per category and add labels
plt.xticks(angles[:-1], categories, color='grey', size=8)

# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([250, 500, 1000], ["250", "500", "1000"], color="grey", size=7)
plt.ylim(0, 1250)

# Plot data
ax.plot(angles, home_health_2020, linewidth=2, linestyle='solid', label='Home Health 2020', color=sns.color_palette('coolwarm', 2)[0])
ax.fill(angles, home_health_2020, sns.color_palette('coolwarm', 2)[0], alpha=0.3)

ax.plot(angles, nursing_2020, linewidth=2, linestyle='solid', label='Nursing 2020', color=sns.color_palette('viridis', 2)[1])
ax.fill(angles, nursing_2020, sns.color_palette('viridis', 2)[1], alpha=0.3)

# Add a title
plt.title('Home Health and Nursing Data for 2020', size=20, color='black', y=1.1)

# Add a legend
plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

# Show the plot
plt.show()